# Statistics of fuji + guadalupe and targets with multiple spectra

April 2022

Angela Berti, Ragadeepika Pucha, Stephanie Juneau, Anthony Kremin<br>
This notebook has been tested with the DESI 23.1 kernel.

### Table of Contents
* [Overview](#overview)
* [Imports](#imports)
* [Accessing the data at NERSC](#data)
* [Catalog statistics by survey](#stats_survey)
  * [Unique targets in each survey](#counts_survey_total)
  * [Survey Validation (SV) versus Main Survey targets](#counts_sv_main_unique)
* [Primary spectra](#primary)
* [Catalog statistics by target type](#stats_target)
  * [DESI target masks](#masks_desi)
  * [Spectra by target type](#counts_target)
* [Redshift distributions by target type and survey](#redshift_dist)
* [Target counts by target type and survey](#counts_target_survey)
* [Targets with multiple spectra](#multispec)
  * [Multiple SPECTYPE classifications](#spectype_diff)
  * [Bright versus dark time spectra of the same target](#bright_dark)

<a class="anchor" id="overview"></a>
## Overview
This notebook is a companion to `Fujilupe_CombineZcat.ipynb`, which demonstrates how combine the [fuji](https://desi.lbl.gov/trac/wiki/Pipeline/Releases/Fuji) and [guadalupe](https://desi.lbl.gov/trac/wiki/Pipeline/Releases/Guadalupe) data releases into a single catalog, flag a unique set of the primary (i.e., best) spectrum for each object in the catalog, and apply some basic quality cuts. This notebook provides examples of how break down the data in the combined catalog by target type (BGS, LRG, etc.), survey (SV, main, etc.), and the number of spectra per target. It also covers how to extract and plot different spectra of the same target.

<a class="anchor" id="imports"></a>
## Imports

In [ ]:
import os
import numpy as np

import fitsio

from astropy.io import fits
from astropy.table import Table, vstack, join
from astropy.convolution import convolve, Gaussian1DKernel

import matplotlib
import matplotlib.pyplot as plt

%matplotlib inline

#-- input/output functions related to DESI spectra
import desispec.io


In [ ]:
#-- specify format of plots

settings = {
    "font.size": 22,
    "axes.linewidth": 2.0,
    "xtick.major.size": 6.0,
    "xtick.minor.size": 4.0,
    "xtick.major.width": 2.0,
    "xtick.minor.width": 1.5,
    "xtick.direction": "in", 
    "xtick.minor.visible": True,
    "xtick.top": True,
    "ytick.major.size": 6.0,
    "ytick.minor.size": 4.0,
    "ytick.major.width": 2.0,
    "ytick.minor.width": 1.5,
    "ytick.direction": "in", 
    "ytick.minor.visible": False,
    "ytick.right": True,
}

plt.rcParams.update(**settings)


<a class="anchor" id="data"></a>
## Accessing the data

The notebook `Fujilupe_CombineZcat.ipynb` shows how this catalog is constructed, and can also be used as a guide to create your own version with just the columns needed for your science case. To load the combined fuji + guadalupe (fujilupe) catalog with *all* available columns:


In [ ]:
%%time

fujilupe = Table( fitsio.read("/global/cfs/cdirs/desi/science/gqp/zcatalog_summary/zall-pix-fujilupe.fits") )


<a class="anchor" id="stats_survey"></a>

## Statistics of the combined catalog by survey

<a class="anchor" id="masks_survey"></a>

### Create masks to select each survey

In [ ]:
#-- main survey
is_main = (fujilupe["SURVEY"].astype(str).data == "main")

#-- SV1/2/3
is_sv1 = (fujilupe["SURVEY"].astype(str).data == "sv1")
is_sv2 = (fujilupe["SURVEY"].astype(str).data == "sv2")
is_sv3 = (fujilupe["SURVEY"].astype(str).data == "sv3")

#-- all SV data
is_sv = (is_sv1 | is_sv2 | is_sv3)

#-- commissioning data
is_cmx = (fujilupe["SURVEY"].astype(str).data == "cmx")

#-- special tiles
is_special = (fujilupe["SURVEY"].astype(str).data == "special")


<a class="anchor" id="counts_survey_total"></a>

### Identify the total number of unique targets observed in each survey, as well as in the combined catalog

Uncomment specific lines to print numbers for additional surveys.

In [ ]:
#-- all unique targets in the main survey
targids_main = np.unique(fujilupe["TARGETID"][is_main])

#-- all unique targets in SV
targids_sv = np.unique(fujilupe["TARGETID"][is_sv])

print("Unique targets in main survey:{0:8}".format( len(targids_main) ))
print("Unique targets in SV:         {0:8}".format( len(targids_sv) ))

print("\nTotal unique targets in combined catalog: {}\n".format( len(np.unique(fujilupe["TARGETID"])) ))

# print("Unique targets in SV1:    {0:8}".format( len(np.unique(fujilupe["TARGETID"][is_sv1])) ))
# print("Unique targets in SV2:    {0:8}".format( len(np.unique(fujilupe["TARGETID"][is_sv2])) ))
# print("Unique targets in SV3:    {0:8}".format( len(np.unique(fujilupe["TARGETID"][is_sv3])) ))

# print("\nUnique targets in CMX:    {0:8}".format( len(np.unique(fujilupe["TARGETID"][is_cmx])) ))

# print("\nUnique targets in Special:{0:8}".format( len(np.unique(fujilupe["TARGETID"][is_special])) ))


Note that the total number of unique targets in the combined catalog is close to, but not exactly, the total of unique targets in the main survey and in SV. This is because some targets are observed in both surveys, and because a (relatively small) number of targets are from commissioning and special tiles.

<a class="anchor" id="counts_sv_main_unique"></a>

### How many unique targets are in only the main survey, only SV, or both?

In [ ]:
print("Unique targets in...\n")
print("  main survey and not SV: {0:8}".format(len(list(set(targids_main).difference(targids_sv))) ))
print("  SV and not main survey: {0:8}".format(len(list(set(targids_sv).difference(targids_main))) ))
print("  both main survey and SV:{0:8}".format(len(list(set(targids_main).intersection(targids_sv))) ))


<a class="anchor" id="primary"></a>

## Primary spectra

The combined catalog contains columns that flag the set of primary spectra within just the main survey (`MAIN_PRIMARY`), just SV data (`SV_PRIMARY`), and the across entire catalog (`ZCAT_PRIMARY`). Note that the set of primary spectra identified within just main survey data or just SV data (`SV_PRIMARY` flag) is *not* the same as identifying primary spectra across the entire combined catalog and *then* selecting by survey, i.e.,

$$
\begin{align}
\texttt{fujilupe}["\!\texttt{ZCAT}\_\texttt{PRIMARY}\!"][\ \texttt{is}\_\texttt{main}\ ] &\neq \texttt{fujilupe}["\!\texttt{MAIN}\_\texttt{PRIMARY}\!"] \\
\texttt{fujilupe}["\!\texttt{ZCAT}\_\texttt{PRIMARY}\!"][\ \texttt{is}\_\texttt{sv}\ ] &\neq \texttt{fujilupe}["\!\texttt{SV}\_\texttt{PRIMARY}\!"] 
\end{align}
$$

<a class="anchor" id="counts_survey_unique"></a>
### How many primary spectra are in the main survey, SV, the combined catalog, and other surveys?
Uncomment specific lines to print numbers for other surveys.

In [ ]:
print("Primary spectra in...\n")
print("  main survey:     {0:8}".format( len(fujilupe[is_main & fujilupe["MAIN_PRIMARY"]]) ))
print("  SV:              {0:8}".format( len(fujilupe[is_sv & fujilupe["SV_PRIMARY"]]) ))
print("  combined catalog:{0:8}\n".format( len(fujilupe[fujilupe["ZCAT_PRIMARY"]]) ))

# print("  SV1:    {0:8}".format( len(fujilupe[is_sv1 & fujilupe["SV_PRIMARY"]]) ))
# print("  SV2:    {0:8}".format( len(fujilupe[is_sv2 & fujilupe["SV_PRIMARY"]]) ))
# print("  SV3:    {0:8}\n".format( len(fujilupe[is_sv3 & fujilupe["SV_PRIMARY"]]) ))

# print("  CMX:    {0:8}\n".format( len(fujilupe[is_cmx & fujilupe["ZCAT_PRIMARY"]]) ))

# print("  Special:{0:8}".format( len(fujilupe[is_special & fujilupe["ZCAT_PRIMARY"]]) ))


As expected, the total number of primary spectra in the combined catalog is exactly the number of unique targets in this catalog (see above).

<a class="anchor" id="stats_target"></a>
## Statistics of the combined catalog by target type

<a class="anchor" id="masks_desi"></a>
### DESI target masks

Targeting bits are encoded in `desi_mask`, which can be imported from `desitarget.targetmask` or from any of the SV libraries (e.g., from `desitarget.sv1.sv1_targetmask`, etc.). While not universally true, for the purposes of this notebook it doesn"t matter which `desi_mask` is used; any of the four import options below will give the same results.


In [ ]:
from desitarget.targetmask import desi_mask, bgs_mask, scnd_mask          # main survey
# from desitarget.sv1.sv1_targetmask import desi_mask, bgs_mask, scnd_mask  # SV1
# from desitarget.sv2.sv2_targetmask import desi_mask, bgs_mask, scnd_mask  # SV2
# from desitarget.sv3.sv3_targetmask import desi_mask, bgs_mask, scnd_mask  # SV3


In [ ]:
#-- select the target mask for each type of target (BGS, LRG, ELG, QSO, etc.)

bgs_tgtmask  = desi_mask["BGS_ANY"]
lrg_tgtmask  = desi_mask["LRG"]
elg_tgtmask  = desi_mask["ELG"]
qso_tgtmask  = desi_mask["QSO"]
mws_tgtmask  = desi_mask["MWS_ANY"]
scnd_tgtmask = desi_mask["SCND_ANY"]


<a class="anchor" id="masks_target"></a>

### Define masks to select all targets of each type from the combined catalog

Information about DESI target selection is stored in the `DESI_TARGET` column. Similarly, the `SV1_DESI_TARGET`, `SV2_DESI_TARGET`, and `SV3_DESI_TARGET` columns indicate which objects were targeted specifically during SV. The masks below select *all* targets of a given type.


In [ ]:
desi_target = fujilupe["DESI_TARGET"]

#-- all BGS targets
is_bgs = (desi_target & bgs_tgtmask != 0) | (fujilupe["SV1_DESI_TARGET"] & bgs_tgtmask != 0) | \
         (fujilupe["SV2_DESI_TARGET"] & bgs_tgtmask != 0) | (fujilupe["SV3_DESI_TARGET"] & bgs_tgtmask != 0)

#-- all LRG targets
is_lrg = (desi_target & lrg_tgtmask != 0) | (fujilupe["SV1_DESI_TARGET"] & lrg_tgtmask != 0) | \
         (fujilupe["SV2_DESI_TARGET"] & lrg_tgtmask != 0) | (fujilupe["SV3_DESI_TARGET"] & lrg_tgtmask != 0)

#-- all ELG targets
is_elg = (desi_target & elg_tgtmask != 0) | (fujilupe["SV1_DESI_TARGET"] & elg_tgtmask != 0) | \
         (fujilupe["SV2_DESI_TARGET"] & elg_tgtmask != 0) | (fujilupe["SV3_DESI_TARGET"] & elg_tgtmask != 0)

#-- all QSO targets
is_qso = (desi_target & qso_tgtmask != 0) | (fujilupe["SV1_DESI_TARGET"] & qso_tgtmask != 0) | \
         (fujilupe["SV2_DESI_TARGET"] & qso_tgtmask != 0) | (fujilupe["SV3_DESI_TARGET"] & qso_tgtmask != 0)

#-- all Milky Way targets
is_mws = (desi_target & mws_tgtmask != 0) | (fujilupe["SV1_DESI_TARGET"] & mws_tgtmask != 0) | \
         (fujilupe["SV2_DESI_TARGET"] & mws_tgtmask != 0) | (fujilupe["SV3_DESI_TARGET"] & mws_tgtmask != 0)

#-- all secondary targets
is_scnd = (desi_target & scnd_tgtmask != 0) | (fujilupe["SV1_DESI_TARGET"] & scnd_tgtmask != 0) | \
          (fujilupe["SV2_DESI_TARGET"] & scnd_tgtmask != 0) | (fujilupe["SV3_DESI_TARGET"] & scnd_tgtmask != 0)


<a class="anchor" id="counts_target"></a>
### Plot the total number of spectra of each target type


In [ ]:
#-- total number of spectra by target type
n_bgs  = len(fujilupe[is_bgs])
n_lrg  = len(fujilupe[is_lrg])
n_elg  = len(fujilupe[is_elg])
n_qso  = len(fujilupe[is_qso])
n_mws  = len(fujilupe[is_mws])
n_scnd = len(fujilupe[is_scnd])

counts = [n_bgs, n_lrg, n_elg, n_qso, n_mws, n_scnd]

#-- number of primary spectra by target type
is_primary = fujilupe["ZCAT_PRIMARY"]

n_bgs_prim  = len(fujilupe[is_bgs & is_primary])
n_lrg_prim  = len(fujilupe[is_lrg & is_primary])
n_elg_prim  = len(fujilupe[is_elg & is_primary])
n_qso_prim  = len(fujilupe[is_qso & is_primary])
n_mws_prim  = len(fujilupe[is_mws & is_primary])
n_scnd_prim = len(fujilupe[is_scnd & is_primary])

counts_prim = [n_bgs_prim, n_lrg_prim, n_elg_prim, n_qso_prim, n_mws_prim, n_scnd_prim]


In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12,6))

targets = ["BGS", "LRG", "ELG", "QSO", "MWS", "SCND"]

ax.bar(targets, counts, color="purple", alpha=0.4, label="All spectra (includes duplicate targets)")
ax.bar(targets, counts_prim, color="purple", alpha=1, label="Primary spectra (unique targets)")
ax.set_ylabel("Number of spectra")
ax.semilogy()
ax.set_ylim(4e5, 2e6)

for i in range(len(targets)):
    ax.text(targets[i], counts[i], counts[i], ha="center", va="bottom", fontsize=18)
    ax.text(targets[i], counts_prim[i], counts_prim[i], ha="center", va="top", fontsize=18, color="white")

plt.legend(fontsize=18, frameon=False, markerfirst=False)
plt.tight_layout()
plt.show()


<a class="anchor" id="redshift_dist"></a>

## Redshift distributions by target type and survey

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16,10))

bins = np.arange(0,4,0.1)

type_masks = (is_bgs, is_lrg, is_elg, is_qso)
colors     = ("C0","C1","C2","C3")

for ax,type_mask,name,color in zip(np.concatenate(axes), type_masks, targets[:-2], colors):
    kwargs = dict(color=color, bins=bins)
    
    ax.hist(fujilupe["Z"][type_mask], **kwargs, alpha=0.3, label=f"Total: {len(fujilupe[type_mask])}")
    ax.hist(fujilupe["Z"][type_mask], bins=bins, histtype="step", color="black")

    mask = type_mask & (is_main | is_sv)
    ax.hist(fujilupe["Z"][mask], **kwargs, alpha=0.6, label=f"SV + Main: {len(fujilupe[mask])}")
    
    mask = type_mask & is_sv
    ax.hist(fujilupe["Z"][mask], **kwargs, label=f"SV only: {len(fujilupe[mask])}")
    ax.hist(fujilupe["Z"][mask], bins=bins, histtype="step", color="black")
    
    ax.legend(fontsize=20, markerfirst=False, handletextpad=0.25, frameon=False)
    ax.set_title(f"{name} targets", fontsize=22)

for ax in axes[-1]:
    ax.set_xlabel("Redshift")

for ax in np.concatenate(axes):
    ax.set_xlim(-0.1,4.0)
    ax.set_xticks(np.arange(0,4.1,0.5))
    ax.set_ylabel("N(z)")
    ax.ticklabel_format(axis="y", style="sci", scilimits=(0,0))

plt.tight_layout()
plt.subplots_adjust(hspace=0.2, wspace=0.15)
plt.show()



### Target class versus Redrock spectral type

The above plot shows $N(z)$ for all *targets* of a given type, but [Redrock](https://github.com/desihub/redrock) will not necessarily classify all spectra as having the same spectral type as their original target class. For example, the following plot breaks down QSO targets by their Redrock spectral type (`GALAXY`, `QSO`, or `STAR`), contained in the `SPECTYPE` column.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10.5,7.5))

bins = np.arange(0,4.1,0.1)

#-- all QSO targets
ax.hist(fujilupe["Z"][is_qso], bins=bins, color="gray", alpha=0.5, label=f"QSO targets: {n_qso}")

#-- QSO targets classified as galaxies
spec_gal = is_qso & (fujilupe["SPECTYPE"]=="GALAXY")
ax.hist(fujilupe["Z"][spec_gal], bins=bins, histtype="step", hatch="\\\\\\", color="k", lw=1.5,
        label=f"SPECTYPE=\"GALAXY\" ({100*len(fujilupe[spec_gal])/n_qso:.1f}%)")

#-- QSO targets classified as quasars
spec_qso = is_qso & (fujilupe["SPECTYPE"]=="QSO")
ax.hist(fujilupe["Z"][spec_qso], bins=bins, color="red", alpha=0.5,
        label=f"SPECTYPE=\"QSO\" ({100*len(fujilupe[spec_qso])/n_qso:.1f}%)")
ax.hist(fujilupe["Z"][spec_qso], bins=bins, histtype="step", color="k", lw=1)

#-- QSO targets classified as stars
spec_star = is_qso & (fujilupe["SPECTYPE"]=="STAR")
ax.hist(fujilupe["Z"][spec_star], bins=bins, color="purple",
        label=f"SPECTYPE=\"STAR\" ({100*len(fujilupe[spec_star])/n_qso:.1f}%)")
ax.hist(fujilupe["Z"][spec_star], bins=bins, histtype="step", color="k", lw=1)

ax.hist(fujilupe["Z"][is_qso], bins=bins, histtype="step", color="black", lw=2)

ax.legend(fontsize=18, markerfirst=False, handletextpad=0.5, frameon=False)

ax.set_xlabel("Redshift")
ax.set_xlim(-0.1,4.0)
ax.set_xticks(np.arange(0,4.1,0.5))
ax.set_ylabel("N(z)")
ax.ticklabel_format(axis="y", style="sci", scilimits=(0,0))

plt.tight_layout()
plt.show()


Of the over 660k QSO targets in the combined catalog, only 49% are actually classified as quasars (red histogram) by Redrock. Nearly 42% are classified as galaxies (black hatched histogram), and the remaining 9% as stars (purple histogram at $z\sim0$). The abrupt drop in $N(z)$ at $z=1.7$ reflects the fact that Redrock is constrained to fit redshifts $z<1.7$ to spectra it classifies as galaxies.

<a class="anchor" id="counts_target_survey"></a>
## Target counts by target type and survey


In [ ]:
#-- BGS
is_bgs_sv1 = (fujilupe["SV1_DESI_TARGET"] & bgs_tgtmask != 0)
is_bgs_sv2 = (fujilupe["SV2_DESI_TARGET"] & bgs_tgtmask != 0)
is_bgs_sv3 = (fujilupe["SV3_DESI_TARGET"] & bgs_tgtmask != 0)
is_bgs_sv  = is_bgs_sv1 | is_bgs_sv2 | is_bgs_sv3 

#-- LRG
is_lrg_sv1 = (fujilupe["SV1_DESI_TARGET"] & lrg_tgtmask != 0)
is_lrg_sv2 = (fujilupe["SV2_DESI_TARGET"] & lrg_tgtmask != 0)
is_lrg_sv3 = (fujilupe["SV3_DESI_TARGET"] & lrg_tgtmask != 0)
is_lrg_sv  = is_lrg_sv1 | is_lrg_sv2 | is_lrg_sv3

#-- ELG
is_elg_sv1 = (fujilupe["SV1_DESI_TARGET"] & elg_tgtmask != 0)
is_elg_sv2 = (fujilupe["SV2_DESI_TARGET"] & elg_tgtmask != 0)
is_elg_sv3 = (fujilupe["SV3_DESI_TARGET"] & elg_tgtmask != 0)
is_elg_sv  = is_elg_sv1 | is_elg_sv2 | is_elg_sv3

#-- QSO
is_qso_sv1 = (fujilupe["SV1_DESI_TARGET"] & qso_tgtmask != 0)
is_qso_sv2 = (fujilupe["SV2_DESI_TARGET"] & qso_tgtmask != 0)
is_qso_sv3 = (fujilupe["SV3_DESI_TARGET"] & qso_tgtmask != 0)
is_qso_sv  = is_qso_sv1 | is_qso_sv2 | is_qso_sv3

#-- Milky Way
is_mws_sv1 = (fujilupe["SV1_DESI_TARGET"] & mws_tgtmask != 0)
is_mws_sv2 = (fujilupe["SV2_DESI_TARGET"] & mws_tgtmask != 0)
is_mws_sv3 = (fujilupe["SV3_DESI_TARGET"] & mws_tgtmask != 0)
is_mws_sv  = is_mws_sv1 | is_mws_sv2 | is_mws_sv3

#-- secondary
is_scnd_sv1 = (fujilupe["SV1_DESI_TARGET"] & scnd_tgtmask != 0)
is_scnd_sv2 = (fujilupe["SV2_DESI_TARGET"] & scnd_tgtmask != 0)
is_scnd_sv3 = (fujilupe["SV3_DESI_TARGET"] & scnd_tgtmask != 0)
is_scnd_sv  = is_scnd_sv1 | is_scnd_sv2 | is_scnd_sv3


In [ ]:
print("BGS    SV:{0:8}\n     Main:{1:8}\n".format(len(fujilupe[is_bgs_sv]), len(fujilupe[is_bgs & ~is_bgs_sv]) ))
print("LRG    SV:{0:8}\n     Main:{1:8}\n".format(len(fujilupe[is_lrg_sv]), len(fujilupe[is_lrg & ~is_lrg_sv]) ))

print("ELG    SV:{0:8}\n     Main:{1:8}\n".format(len(fujilupe[is_elg_sv]), len(fujilupe[is_elg & ~is_elg_sv]) ))
print("QSO    SV:{0:8}\n     Main:{1:8}\n".format(len(fujilupe[is_qso_sv]), len(fujilupe[is_qso & ~is_qso_sv]) ))

print("MWS    SV:{0:8}\n     Main:{1:8}\n".format(len(fujilupe[is_mws_sv]), len(fujilupe[is_mws & ~is_mws_sv]) ))
print("SCND   SV:{0:8}\n     Main:{1:8}\n".format(len(fujilupe[is_scnd_sv]), len(fujilupe[is_scnd & ~is_scnd_sv]) ))


<a class="anchor" id="multispec"></a>
## Targets with multiple spectra

As shown in the plot below, the vast majority of targets in the combined catalog have only one spectrum. About 3% of targets have two spectra, and only a few thousand have three, four, or five (the maximum).

Note that these numbers will be slightly different depending on whether the bad fiber quality cut has been applied to the combined catalog. If spectra associated with bad fibers are removed *after* `ZCAT_NSPEC` has been computed and added to the catalog, then some targets with `ZCAT_NSPEC`$\,=x$ (where $x>1$) may appear in the catalog fewer than $x$ times.


In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8,6))

freq, hist = np.array(np.unique(fujilupe["ZCAT_NSPEC"], return_counts=True))
hist  = ( hist / np.arange(1,np.max(freq)+1) ).astype(int)
n_tot = sum(hist)
ax.bar(freq, hist, color="purple", alpha=0.4)

ax.set_xlabel("Number of spectra per target")
ax.set_xlim(0.4,5.6)
ax.set_ylabel("Number of unique targets")
ax.set_ylim(1e1,1e8)
ax.semilogy()

round_to = [1,1,2,2,3]
pct = [f"{np.round( 100*hist[i]/n_tot, round_to[i] )}" for i in range(len(round_to))]

for i in range(len(round_to)):
    ax.text(freq[i], hist[i], f"{hist[i]}\n({pct[i]}%)", ha="center", va="bottom", fontsize=18)

plt.tight_layout()
plt.show()


#### Below we'll look more closely at the targets with five spectra


In [ ]:
#-- get all targets with NSPEC=5
t_fivespec = fujilupe[fujilupe["ZCAT_NSPEC"]==5]

#-- unique TARGETID of each object with five spectra
targids = np.unique(t_fivespec["TARGETID"])

print("\tTARGETID\t\tSPECTYPE of all 5 spectra")
for i,tid in enumerate(targids):
    these_spec = t_fivespec[t_fivespec["TARGETID"]==tid]
    spectype   = these_spec["SPECTYPE"].data.astype(str)
    print("{0:3}\t{1}\t{2}".format(i+1, tid, spectype))


Most of the targets with five spectra have been flagged as stars by Redrock. One object (12 above) has four spectra flagged as a star, but the fifth classification is a galaxy. Let's look more closely at this object.

<a class="anchor" id="spectype_diff"></a>
### Targets with multiple `SPECTYPE` classifications


In [ ]:
#-- get the data for all observations of this TARGETID
these_spec = t_fivespec[ t_fivespec["TARGETID"]==targids[11] ]

#-- get the SURVEY, PROGRAM, SPECTYPE, and redshift values for each of the five spectra of this object
survey   = these_spec["SURVEY"].data.astype(str)
program  = these_spec["PROGRAM"].data.astype(str)
redshift = np.round(these_spec["Z"].data,5)
spectype = these_spec["SPECTYPE"].data.astype(str)

print("\tSURVEY  PROGRAM  SPECTYPE  REDSHIFT")
for i in range(5):
    print("{0:1}/5\t{1:7} {2:8} {3:8} {4:8}".format(i+1, survey[i], program[i], spectype[i], redshift[i]))


The four spectra classified as `STAR` have $z\sim0$, while the redshift of the spectrum with the `GALAXY` classification is $z\sim0.316$. Let's plot several of these spectra to invesitgate the different in `SPECTYPE` classification and redshift measurement.

The `get_spec_data` function defined below is a convenience function to load the relevant data for plotting the spectrum for a given `TARGETID`, `SURVEY`, and `PROGRAM`. We will use it to load different spectra of the same object.

In [ ]:
def get_spec_data(tid, survey=None, program=None):
    #-- the index of the specific target can be uniquely determined with the combination of TARGETID, SURVEY, and PROGRAM
    idx = np.where( (fujilupe["TARGETID"]==tid) & (fujilupe["SURVEY"]==survey) & (fujilupe["PROGRAM"]==program) )[0][0]

    #-- healpix values are integers but are converted here into a string for easier access to the file path
    hpx = fujilupe["HEALPIX"].astype(str)

    if "sv" in survey:
        specprod = "fuji"
    elif survey=="main":
        specprod = "guadalupe"

    specprod_dir = f"/global/cfs/cdirs/desi/spectro/redux/{specprod}"
    target_dir   = f"{specprod_dir}/healpix/{survey}/{program}/{hpx[idx][:-2]}/{hpx[idx]}"
    coadd_fname  = f"coadd-{survey}-{program}-{hpx[idx]}.fits"

    #-- read in the spectra with desispec
    coadd_obj  = desispec.io.read_spectra(f"{target_dir}/{coadd_fname}")
    coadd_tgts = coadd_obj.target_ids().data

    #-- select the spectrum of  targetid
    row = ( coadd_tgts==fujilupe["TARGETID"][idx] )
    coadd_spec = coadd_obj[row]

    return coadd_spec


In [ ]:
#-- get the third, fourth, and fifth spectra for this object 

tid = targids[11] #-- TARGETID

#-- 3/5 above; SPECTYPE = STAR
sv1_other_star = get_spec_data(tid, survey="sv1", program="other")

#-- 4/5 above; SPECTYPE = STAR
sv3_bright_star = get_spec_data(tid, survey="sv3", program="bright")

#-- 5/5 above; SPECTYPE = GALAXY
sv3_dark_gal = get_spec_data(tid, survey="sv3", program="dark")


In [ ]:
#-- plot the spectra

fig, ax = plt.subplots(1, 1, figsize=(18,6))

ax.set_xlim(3500, 9900)
ax.set_xlabel(r"wavelength $\lambda\ \left[ \AA \right]$")
ax.set_ylim(-2, 44)
ax.set_ylabel(r"$F_{\lambda}\ \left[ 10^{-17}\ {\rm erg\ s}^{-1}\ {\rm cm}^{-2}\ \AA^{-1} \right]$")

spectra = ( sv1_other_star, sv3_dark_gal, sv3_bright_star )
zwarn   = (0,0,1570)
coaddfs = (0,0,0)
for PROG,ST,spec,color,zw,cfs in zip( ("OTHER","DARK","BRIGHT"), ("STAR","GALAXY","STAR"), spectra, ("k","orange","magenta"), zwarn, coaddfs ):
    for band in ("b","r","z"):
        label = f"PROG={PROG} | SPECTYPE={ST} | COADD_FIBERSTATUS={cfs} | ZWARN={zw}" if band=="b" else ""
        ax.plot(spec.wave[band], convolve(spec.flux[band][0], Gaussian1DKernel(5)), label=label, color=color, lw=2)

ax.text(3600, 39, f"TARGETID = {tid}", ha="left", va="bottom")
ax.legend(markerfirst=False, fontsize=18, handletextpad=0.5, frameon=False)

#-- annotations
ax.text(5800, 10, r"$\leftarrow$" + "arm overlap (not absorption)" + r"$\rightarrow$", ha="left", va="bottom", fontsize=19, color="red", rotation=350)
ax.text(4800, 34, r"$\downarrow$", ha="left", va="bottom", fontsize=24, color="red")
ax.text(6500, 23, r"$\downarrow$", ha="left", va="bottom", fontsize=24, color="red")
ax.text(4920, 36, "absorption line", ha="left", va="bottom", fontsize=19, color="red")
ax.text(6620, 25, "absorption line", ha="left", va="bottom", fontsize=19, color="red")

plt.tight_layout()
plt.show()


The SV1 dark time spectrum (black line) is clearly a stellar spectrum and correctly classified by Redrock, while the shape of the SV3 "other" program spectrum (orange line) is somewhat ambiguous. It is not surpising that this spectrum was misclassified as a galaxy. The SV3 bright time spectrum (magenta line) obviously has no flux data, and it is not immediately clear why it was classified as a stellar spectrum. Below we'll look at additional columns for these spectra.


In [ ]:
#-- show all data columns for the five spectra for this target

t_fivespec[ t_fivespec["TARGETID"]==tid ].show_in_notebook()


Looking at all the columns for these five spectra, we can see that all five have `COADD_FIBERSTATUS=0` (i.e., no bad fiber warning). However, the spectrum with no flux (4/5) has `ZWARN=1570`, while the other (good) spectra all have `ZWARN=0`. Scroll to the final column (`ZCAT_PRIMARY`) and we'll see that the third spectra (black line in the above plot) is the primary spectrum for this target.

<a class="anchor" id="bright_dark"></a>
### `BRIGHT` versus `DARK` program spectra of the same target

Now we'll look more closely at the `SURVEY`, `PROGRAM`, and redshift values of the four galaxy targets with five spectra.

In [ ]:
for tid in targids[:4]:
    these_spec = t_fivespec[t_fivespec["TARGETID"]==tid]
    survey   = these_spec["SURVEY"].data.astype(str)
    program  = these_spec["PROGRAM"].data.astype(str)
    redshift = these_spec["Z"].data
    print(f"TARGETID = {tid}")
    print("    Survey  \t{0}\t{1}\t{2}\t{3}\t{4}".format(*survey))
    print("    Program \t{0}\t{1}\t{2}\t{3}\t{4}".format(*program))
    print("    Redshift\t{0:.4f}\t{1:.4f}\t{2:.4f}\t{3:.4f}\t{4:.4f}\n".format(*redshift))


Each of these targets was observed once each during SV1, SV2, and SV3, and twice during the main survey (once each during the bright and dark programs). Note that for each object all five redshift measurements agree to at least the third or fourth decimal place.

Below we compare the dark and bright time spectra of this object from the main survey. The top panel shows the smoothed bright time spectrum in magenta against the smoothed dark time spectrum in black. The middle and bottom panels show the bright and dark time spectra, respectively, smoothed in black, as well as the flux from each arm (*B*, *R*, and *Z*) shown in blue, green, and red. As expected, the bright time spectrum is clearly nosier than the dark time. While the two spectra have very similar continuua, the $4000$ Angstrom break (gray dashed line; redshifted to $\sim5600\ {\rm A}$) is easier to identify in the dark time sepctrum.

In [ ]:
#-- get two spectra for this object (first of the four galaxies with five spectra)

tid = targids[0] #-- TARGETID 

#-- get the DARK TIME spectrum
spec_dark = get_spec_data(tid, survey="main", program="dark")

#-- get the BRIGHT TIME spectrum
spec_bright = get_spec_data(tid, survey="main", program="bright")


In [ ]:
#-- plot the spectra

fig, axes = plt.subplots(3, 1, figsize=(18,15), sharex=True, sharey=True)

for ax in axes:
    ax.set_xlim(3500, 9900)
    ax.set_ylim(-3.9, 6)  #-- change depending on specific spectrum
    ax.set_ylabel(r"$F_{\lambda}\ \left[ 10^{-17}\ {\rm erg\ s}^{-1}\ {\rm cm}^{-2}\ \AA^{-1} \right]$")
    ax.plot(5610*np.ones(2), ax.get_ylim(), color="gray", ls="--", label="$4000\ \AA$ break (approx.)")

axes[-1].set_xlabel(r"wavelength $\lambda\ \left[ \AA \right]$")

#-- plot the BRIGHT TIME and DARK TIME spectra (both smoothed) on the same axis
for band in ("b","r","z"):
    if band=="z":
        label_bright = "PROGRAM = BRIGHT"
        label_dark   = "PROGRAM = DARK"
    else:
        label_bright = label_dark = ""
    axes[0].plot(spec_bright.wave[band], convolve(spec_bright.flux[band][0], Gaussian1DKernel(5)), color="magenta", label=label_bright)
    axes[0].plot(spec_dark.wave[band], convolve(spec_dark.flux[band][0], Gaussian1DKernel(5)), color="k", label=label_dark)
axes[0].legend(markerfirst=False, loc=4, handletextpad=0.5, frameon=False)
axes[0].text(3600, 0.8*axes[0].get_ylim()[1], f"TARGETID = {tid}", ha="left", va="bottom")

#-- plot the BRIGHT TIME and DARK TIME spectra at full resolution on separate axes
for ax,coadd_spec,prog in zip(axes[1:],(spec_bright,spec_dark),("BRIGHT","DARK")):
    ax.text(9800, 0.9*ax.get_ylim()[0], f"\nPROGRAM = {prog.upper()}", ha="right", va="bottom")
    for band,color in zip(("b","r","z"),("b","g","r")):
        #-- plot the spectrum from each arm (B,R,Z) in blue, green, red
        ax.plot(coadd_spec.wave[band], coadd_spec.flux[band][0], color=color, alpha=0.3)
        #-- add smoothed spectrum in black for all the three arms
        ax.plot(coadd_spec.wave[band], convolve(coadd_spec.flux[band][0], Gaussian1DKernel(5)), color="k")
        
plt.tight_layout()
plt.subplots_adjust(hspace=0.05)
plt.show()
